### Exercicio 1:


1.  Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    a. Cada reunião ocupa uma sala (enumeradas $$1...S\,$$) durante um “slot” $$(\text{tempo},\text{dia})$$.  Assume-se os dias enumerados $$1..D$$ e, em cada dia, os tempos enumerados $$1..T$$.
    b.  Cada reunião tem associado um projeto (enumerados $$1..P$$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $$1..C$$.
    c. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
    d. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $$50\%$$ do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema). 

## Inputs do Problema:

S       - Número de salas \
D       - Dias \
T       - Tempo \
P       - Número de projeto \
C       - Número de colaboradores \
(T,D)   - Formato de um slot (tempo,dia)

slots   - Conjunto de slots da semana

Sendo *colaboradores* o conjunto de todos os colaboradores, *colaboradores*<sub>**i**</sub> define todos os slots em que o colaborador está disponível.

Sendo *projetos* o conjunto de todos os projetos, *projetos*<sub>**i**</sub> define o número de reuniões semanais do projeto, a lista dos colaboradores do projeto e um colaborador do projeto como líder do mesmo.

## Restrições da construção do horário

1. O líder têm que participar em todas as reuniões do projeto
1. Todas as reuniões de um dado projeto têm de ter uma participação mínima de 50% dos colaboradores
1. Uma dada sala só pode ter uma e uma só reunião por slot
1. Um colaborador não pode participar em mais de uma reunião por slot
1. Todos os projetos tem um dado número de reuniões
1. Todos os colaboradores tem um conjunto de slots em que estão disponíveis
1. Um colaborador não pode participar numa reunião fora da sua disponibilidades
1. Um colaborador não pode participar numa reunião de um projeto ao qual não está alocado

In [5]:
%pip install ortools
%pip install prettytable

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [212]:
### Completar
from ortools.linear_solver import pywraplp
from prettytable import PrettyTable
import random

random.seed()

S = 4
D = 5
T = 5
P = 5
C = 20

slots = []
colaboradores = {} # colaboradores[i] = [s] - [s] são slots em que o colaborador está disponível

projetos = {} # projetos[i] = (l,r,[c]) - l é o lider do projeto
              #                         - r é o número de reuniões do projeto                  
              #                         - [c] são os colaboradores associados ao projeto

#horarios = []

#for d in range(0,D):
 #   header = []
  #  temp = "Dia " + str(d)
   # header.append(temp)
    #for t in range(0,T):
     #   header.append(t)
      #  slots.append((t+1,d+1)
       # horarios.append(PrettyTable(header))
header = ["Dia %i" % d for d in range(1,D+1)]
header.insert(0,"Tempos")

for d in range(0,D):
    for t in range(0,T):
        slots.append((t+1,d+1))

#gerar exemplo
# 1: Disponibilidade dos Colaboradores
for c in range(1,C+1):
    d = random.randrange(15,20)
    s = 0
    disp = []
    while s < d:
        choice = random.choice(slots)
        if choice not in disp:
            disp.append(choice)
            s+=1
    colaboradores[c] = disp

# 2: Criação dos projetos
for p in range(1,P+1):
    r = random.randrange(4,8)
    nc = random.randrange(3,6)
    i = 0
    colabs = []
    while i < nc:
        choice = random.choice([1,2,3,4,5,6,7,8,9,10])
        if choice not in colabs:
            colabs.append(choice)
            i+=1
    lider = random.choice(colabs)
    projetos[p] = (lider,r,colabs)

tabelaColaboradores = PrettyTable(header)

for t in range(1,T+1):
        slots = []
        slots.append(("Tempo %i" % t))
        for d in range(1,D+1):
            slot = ""
            for c in range(1,C+1):
                for x,y in colaboradores[c]:
                    if x == t and y == d:
                        slot += ("%i," % c)
            slot = slot[:-1]
            slots.append(slot)
        tabelaColaboradores.add_row(slots)
        
print("Disponibilidade Colaboradores:")
print(tabelaColaboradores)
print("\n")

tabelaProjetos = PrettyTable(["Projeto","Líder","Nº de reuniões","Colaboradores"])

for p in projetos:
    tabelaProjetos.add_row([p,projetos[p][0],projetos[p][1],projetos[p][2]])

print("Projetos:")
print(tabelaProjetos)
print("\n")

alocaSalas = {} # Matriz que guarda se uma dada sala s está atribuida a um projeto p num dado slot (t,d)
alocaColab = {} # Matriz que guarda se um dado colaborador está atribuido a um projeto p num dado slot (t,d)

# inicializar alocaSalas e alocaColab

solver = pywraplp.Solver.CreateSolver('SCIP')

for s in range(1,S+1):
    alocaSalas[s] = {}
    for d in range(1,D+1):
        alocaSalas[s][d] = {}
        for t in range(1,T+1):
            alocaSalas[s][d][t] = {}
            for p in range(1,P+1):
                alocaSalas[s][d][t][p] = solver.BoolVar('alocaSalas[%i][%i][%i][%i]' % (s,d,t,p))

for c in range(1,C+1):
    alocaColab[c] = {}
    for d in range(1,D+1):
        alocaColab[c][d] = {}
        for t in range(1,T+1):
            alocaColab[c][d][t] = {}
            for p in range(1,P+1):
                alocaColab[c][d][t][p] = solver.BoolVar('alocaColab[%i][%i][%i][%i]' % (c,d,t,p))

# O líder têm que participar em todas as reuniões do projeto

for d in range(1,D+1):
    for t in range(1,T+1):
        for p in range(1,P+1):
            l = projetos[p][0]
            solver.Add(sum(alocaSalas[s][d][t][p] for s in range(1,S+1)) == alocaColab[l][d][t][p])

# Uma sala só pode ter uma reunião por slot

for s in range(1,S+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            solver.Add(sum(alocaSalas[s][d][t][p] for p in range(1,P+1)) <= 1)

# Um colaborador não pode ser alocado a uma reunião cujo slot está fora da disponibilidade do colaborador

for c in range(1,C+1):
    for p in range(1,P+1):
        for d in range(1,D+1):
            for t in range(1,T+1):
                if (t,d) not in colaboradores[c]:
                    solver.Add(alocaColab[c][d][t][p] == 0)

# Um colaborador não pode participar em mais de uma reunião no mesmo slot

for c in range(1,C+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            solver.Add(sum(alocaColab[c][d][t][p] for p in range(1,P+1)) <= 1)

# Um projeto tem que ter um dado número de reuniões

for p in range(1,P+1):
    solver.Add(sum(alocaSalas[s][d][t][p] for s in range(1,1+S) for d in range(1,D+1) for t in range(1,T+1)) == projetos[p][1])

# Uma reunião têm que ter no mínimo uma presença de 50% dos colaboradores

for p in range(1,P+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            l = projetos[p][0]
            solver.Add(sum(alocaColab[c][d][t][p] for c in range(1,C+1)) >= 0.5*len(projetos[p][2])*alocaColab[l][d][t][p])

# Um colaborador não pode participar numa reunião de um projeto ao qual não está alocado

for c in range(1,C+1):
    for p in range(1,P+1):
        if c not in projetos[p][2]:
            solver.Add(sum(alocaColab[c][d][t][p] for d in range(1,D+1) for t in range(1,T+1)) == 0)

# Solução para o problema
r = solver.Solve()

if r == pywraplp.Solver.OPTIMAL:

    horario = PrettyTable(header)

    for t in range(1,T+1):
        slots = []
        slots.append(("Tempo %i" % t))
        for d in range(1,D+1):
            slot = ""
            for s in range(1,S+1):
                for p in range(1,P+1):
                    if round(alocaSalas[s][d][t][p].solution_value()) == 1:
                        slot += ("Sala %i - Projeto %i \n Colab: " % (s,p))
                        for c in projetos[p][2]:
                            if round(alocaColab[c][d][t][p].solution_value()) == 1:
                                slot += ("%i " %c)
                        slot += ("\n")
            slots.append(slot)
        horario.add_row(slots, divider=True)
    
    print("Solução:")
    print(horario)

else:
    print("Solução não encontrada")


Disponibilidade Colaboradores:
+---------+-----------------------------------------+--------------------------------------------+---------------------------------------+----------------------------------------+----------------------------------------+
|  Tempos |                  Dia 1                  |                   Dia 2                    |                 Dia 3                 |                 Dia 4                  |                 Dia 5                  |
+---------+-----------------------------------------+--------------------------------------------+---------------------------------------+----------------------------------------+----------------------------------------+
| Tempo 1 |        1,2,4,6,7,9,10,12,15,17,19       |      1,2,3,5,7,9,11,13,14,16,17,18,20      |  1,3,4,5,6,7,8,9,10,11,12,13,15,18,20 | 1,2,4,5,7,8,9,10,11,12,13,15,16,18,20  |   1,4,6,8,9,10,11,13,14,15,16,17,18    |
| Tempo 2 |      2,4,5,6,7,8,10,11,15,16,19,20      |  1,3,4,5,6,10,12,13,14,15,16,17